# 1 er TEST

## Primer test, análisis sencillo para probar modelos y tener una primera aprox


### En esta primera aproximación tratamos los datos "raros" como NaN, los llenamos con mediana o moda en función del tipo de la etiqueta, hacemos un split aleatorio y lanzamos un par de modelos para ver cual funciona mejor y como


In [31]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

df_raw = pd.read_csv("train_IA_PROJECT.csv")
df_raw.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51.0,1.0,1.0,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0
1,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,-9.0,-9.0,7.0,2
2,63.0,1.0,4.0,140,0,?,2.0,149,0,2,1,?,?,2
3,52.0,0.0,2.0,140.0,-9.0,0.0,0.0,140.0,0.0,0.0,-9.0,-9.0,-9.0,0
4,55.0,1.0,4.0,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3


In [32]:
#Reemplazamos ? por nan para poder tratarlos, nos aseguramos de que las variables numericas lo son 
# y quitamos 0 que no pueden ser en colesterol y presion arterial. Separamos variables num
# de variables categoricas

df = df_raw.copy()


df = df.replace("?", np.nan)


num_cols_all = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg',
                'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal', 'label']

for col in num_cols_all:
    df[col] = pd.to_numeric(df[col], errors='coerce')


df.loc[df["trestbps"] == 0, "trestbps"] = np.nan
df.loc[df["chol"] == 0, "chol"] = np.nan


num_cols = ["age", "trestbps", "chol", "thalach", "oldpeak"]
cat_cols = ["sex", "cp", "fbs", "restecg", "exang", "slope", "ca", "thal"]
target_col = "label"

df.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label
0,51.0,1.0,1.0,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0
1,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,-9.0,-9.0,7.0,2
2,63.0,1.0,4.0,140.0,NaN,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2
3,52.0,0.0,2.0,140.0,-9.0,0.0,0.0,140.0,0.0,0.0,-9.0,-9.0,-9.0,0
4,55.0,1.0,4.0,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3


In [33]:
#Sustituimos los -9 con nan
df0 = df.copy()

cols_neg9 = ["chol", "fbs", "slope", "ca", "thal"]

for col in cols_neg9:
    df0.loc[df0[col] == -9, col] = np.nan

df0[cat_cols + num_cols].head()


,sex,cp,fbs,restecg,exang,slope,ca,thal,age,trestbps,chol,thalach,oldpeak
0,1.0,1.0,0.0,2.0,1.0,1.0,1.0,3.0,51.0,125.0,213.0,125.0,1.4
1,1.0,3.0,0.0,0.0,1.0,NaN,NaN,7.0,54.0,120.0,237.0,150.0,1.5
2,1.0,4.0,NaN,2.0,0.0,1.0,NaN,NaN,63.0,140.0,NaN,149.0,2.0
3,0.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,52.0,140.0,NaN,140.0,0.0
4,1.0,4.0,0.0,0.0,1.0,3.0,0.0,7.0,55.0,140.0,217.0,111.0,5.6


In [34]:
# Separamos etiquetas de objetivo
X0 = df0[num_cols + cat_cols]
y0 = df0[target_col]

#Utilizamos la mediana para llenar los nan de los numeros
imp_num_0 = SimpleImputer(strategy="median")
X0[num_cols] = imp_num_0.fit_transform(X0[num_cols])

# Utilizamos la moda para las categoricas
imp_cat_0 = SimpleImputer(strategy="most_frequent")
X0[cat_cols] = imp_cat_0.fit_transform(X0[cat_cols])

X0.isna().sum()


/var/folders/mt/jht7x_zx7q74g1bmh0zcxgt40000gn/T/ipykernel_72971/1471085634.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X0[num_cols] = imp_num_0.fit_transform(X0[num_cols])
/var/folders/mt/jht7x_zx7q74g1bmh0zcxgt40000gn/T/ipykernel_72971/1471085634.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X0[cat_cols] = imp_cat_0.fit_transform(X0[cat_cols])


age         0
trestbps    0
chol        0
thalach     0
oldpeak     0
sex         0
cp          0
fbs         0
restecg     0
exang       0
slope       0
ca          0
thal        0
dtype: int64

In [35]:
#Hacemos un split aleatorio poniendo un 0,2 para test
X0_train, X0_test, y0_train, y0_test = train_test_split(
    X0, y0, test_size=0.2, stratify=y0, random_state=42
)


In [ ]:
#Aplicamos regresion logistica, el mejor iter = 200
log_reg_0 = LogisticRegression(
    multi_class="multinomial",
    max_iter=200
    ,
    n_jobs=-1
)

log_reg_0.fit(X0_train, y0_train)
y0_pred_lr = log_reg_0.predict(X0_test)

print("=== Estrategia 0 - Logistic Regression ===")
print("Accuracy:", accuracy_score(y0_test, y0_pred_lr))
print("Macro F1:", f1_score(y0_test, y0_pred_lr, average="macro"))
print(confusion_matrix(y0_test, y0_pred_lr))


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


=== Estrategia 0 - Logistic Regression ===
Accuracy: 0.5374149659863946
Macro F1: 0.3372599949070537
[[58  7  1  0  0]
 [17 11  0  3  0]
 [ 7  7  4  4  0]
 [ 6  8  1  6  0]
 [ 0  4  2  1  0]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [71]:
# Aplicamos RandomForest, mejor n_estimators = 400
rf_0 = RandomForestClassifier(
    n_estimators=400,
    random_state=42
)

rf_0.fit(X0_train, y0_train)
y0_pred_rf = rf_0.predict(X0_test)

print("=== Estrategia 0 - Random Forest ===")
print("Accuracy:", accuracy_score(y0_test, y0_pred_rf))
print("Macro F1:", f1_score(y0_test, y0_pred_rf, average="macro"))
print(confusion_matrix(y0_test, y0_pred_rf))




=== Estrategia 0 - Random Forest ===
Accuracy: 0.5170068027210885
Macro F1: 0.3519589771746804
[[56  6  2  2  0]
 [17  9  1  3  1]
 [ 7  7  2  4  2]
 [ 3  6  4  8  0]
 [ 3  2  1  0  1]]


# 2º TEST


## Vamos a comparar los dos modelos anteriores con un EDA más completo


### Ahora con los -9 tendremos en cuenta nuestra hipotesis de que no se han realizado más pruebas, añadiendo esas columnas, rellenaremos con KNN las numericas y con la moda las categoricas.

In [ ]:
#Creamos las columnas que indican que pruebas se les han hecho a los pacientes y 
# sustituimos en las originales por NaN para rellenarlos
df1 = df.copy()

cols_neg9 = ["chol", "fbs", "slope", "ca", "thal"]

for col in cols_neg9:
    
    df1[col + "_not_done"] = (df1[col] == -9).astype(int)
    
    df1.loc[df1[col] == -9, col] = np.nan

df1.head()


,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,label,chol_not_done,fbs_not_done,slope_not_done,ca_not_done,thal_not_done
0,51.0,1.0,1.0,125.0,213.0,0.0,2.0,125.0,1.0,1.4,1.0,1.0,3.0,0,0,0,0,0,0
1,54.0,1.0,3.0,120.0,237.0,0.0,0.0,150.0,1.0,1.5,NaN,NaN,7.0,2,0,0,1,1,0
2,63.0,1.0,4.0,140.0,NaN,NaN,2.0,149.0,0.0,2.0,1.0,NaN,NaN,2,0,0,0,0,0
3,52.0,0.0,2.0,140.0,NaN,0.0,0.0,140.0,0.0,0.0,NaN,NaN,NaN,0,1,0,1,1,1
4,55.0,1.0,4.0,140.0,217.0,0.0,0.0,111.0,1.0,5.6,3.0,0.0,7.0,3,0,0,0,0,0


In [52]:
# Con KNN rellenamos variables numericas

from sklearn.impute import KNNImputer
knn_imputer = KNNImputer(n_neighbors=5)
df1[num_cols] = knn_imputer.fit_transform(df1[num_cols])

# Rellenamos con la moda las categoricas
for col in cat_cols:
    df1[col].fillna(df1[col].mode()[0], inplace=True)

# Definimos x e y
extra_not_done = [c + "_not_done" for c in cols_neg9]
X1 = df1[num_cols + cat_cols + extra_not_done]
y1 = df1[target_col]

X1.isna().sum()


/var/folders/mt/jht7x_zx7q74g1bmh0zcxgt40000gn/T/ipykernel_72971/2902982829.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df1[col].fillna(df1[col].mode()[0], inplace=True)


age               0
trestbps          0
chol              0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
fbs_not_done      0
slope_not_done    0
ca_not_done       0
thal_not_done     0
dtype: int64

In [53]:
# De nuevo hacemos un split aleatorio
X1_train, X1_test, y1_train, y1_test = train_test_split(
    X1, y1, test_size=0.2, stratify=y1, random_state=42
)


In [64]:
# Volvemos a aplicar Regresion Logistica
log_reg_1 = LogisticRegression(
    multi_class="multinomial",
    max_iter=200,
    n_jobs=-1
)

log_reg_1.fit(X1_train, y1_train)
y1_pred_lr = log_reg_1.predict(X1_test)

print("=== Estrategia 1 - Logistic Regression ===")
print("Accuracy:", accuracy_score(y1_test, y1_pred_lr))
print("Macro F1:", f1_score(y1_test, y1_pred_lr, average="macro"))
print(confusion_matrix(y1_test, y1_pred_lr))


=== Estrategia 1 - Logistic Regression ===
Accuracy: 0.5306122448979592
Macro F1: 0.3333640514736405
[[56  9  1  0  0]
 [13 13  1  4  0]
 [ 5 10  4  3  0]
 [ 5  9  1  5  1]
 [ 1  3  3  0  0]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 200 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=200).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Aplicamos de nuevo Random Forest, para este caso n =400 es peor y n = 500 mejora 
rf_1 = RandomForestClassifier(
    n_estimators=500,
    random_state=42
)

rf_1.fit(X1_train, y1_train)
y1_pred_rf = rf_1.predict(X1_test)

print("=== Estrategia 1 - Random Forest ===")
print("Accuracy:", accuracy_score(y1_test, y1_pred_rf))
print("Macro F1:", f1_score(y1_test, y1_pred_rf, average="macro"))
print(confusion_matrix(y1_test, y1_pred_rf))


=== Estrategia 1 - Random Forest ===
Accuracy: 0.4965986394557823
Macro F1: 0.3396347581778045
[[55  7  3  1  0]
 [16  8  3  3  1]
 [ 9  5  4  3  1]
 [ 3  7  6  5  0]
 [ 2  2  2  0  1]]


### Si lo comparamos con el primer metodo vemos que añadir columnas con este tipo de rellenado y para estos dos modelos no parece la mejor opcion, ya que ambos empeoran

# 3r TEST

## Ahora para seguir afinando y comparamos MICE vs KNN para rellenar los NaN

### En este caso vamos a seguir el mismo preprocesado de antes pero ahora llenaremos los NaN con MICE y compararemos

In [ ]:
# Volvemos a crear columnas para identificar que pruebas se han o no hecho y a rellenar 
# .9, ? y 0 con NaN para luego sustituirlos
df2 = df.copy()

cols_neg9 = ["chol", "fbs", "slope", "ca", "thal"]
for col in cols_neg9:
    df2[col + "_not_done"] = (df2[col] == -9).astype(int)
    df2.loc[df2[col] == -9, col] = np.nan

# Definimos las columnas que vamos a imputar con MICE
features_2 = num_cols + cat_cols + [c + "_not_done" for c in cols_neg9]
df2[features_2].head()


,age,trestbps,chol,thalach,oldpeak,sex,cp,fbs,restecg,exang,slope,ca,thal,chol_not_done,fbs_not_done,slope_not_done,ca_not_done,thal_not_done
0,51.0,125.0,213.0,125.0,1.4,1.0,1.0,0.0,2.0,1.0,1.0,1.0,3.0,0,0,0,0,0
1,54.0,120.0,237.0,150.0,1.5,1.0,3.0,0.0,0.0,1.0,NaN,NaN,7.0,0,0,1,1,0
2,63.0,140.0,NaN,149.0,2.0,1.0,4.0,NaN,2.0,0.0,1.0,NaN,NaN,0,0,0,0,0
3,52.0,140.0,NaN,140.0,0.0,0.0,2.0,0.0,0.0,0.0,NaN,NaN,NaN,1,0,1,1,1
4,55.0,140.0,217.0,111.0,5.6,1.0,4.0,0.0,0.0,1.0,3.0,0.0,7.0,0,0,0,0,0


In [ ]:
#Aplicamos MICE para rellenar todos los NaNs
from sklearn.impute import IterativeImputer

imp_iter = IterativeImputer(random_state=42, max_iter=15)
df2[features_2] = imp_iter.fit_transform(df2[features_2])

X2 = df2[features_2]
y2 = df2[target_col]

X2.isna().sum()


age               0
trestbps          0
chol              0
thalach           0
oldpeak           0
sex               0
cp                0
fbs               0
restecg           0
exang             0
slope             0
ca                0
thal              0
chol_not_done     0
fbs_not_done      0
slope_not_done    0
ca_not_done       0
thal_not_done     0
dtype: int64

In [ ]:
# Split de trains
X2_train, X2_test, y2_train, y2_test = train_test_split(
    X2, y2, test_size=0.2, stratify=y2, random_state=42
)


In [ ]:
#Aquí si que sí podemos apreciar una mejora aplicando la regresion logistica para n=400 como en los
# casos anteriores 
log_reg_2 = LogisticRegression(
    multi_class="multinomial",
    max_iter=400,
    n_jobs=-1
)

log_reg_2.fit(X2_train, y2_train)
y2_pred_lr = log_reg_2.predict(X2_test)

print("=== Estrategia 2 - Logistic Regression (MICE) ===")
print("Accuracy:", accuracy_score(y2_test, y2_pred_lr))
print("Macro F1:", f1_score(y2_test, y2_pred_lr, average="macro"))
print(confusion_matrix(y2_test, y2_pred_lr))


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:1272: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.8. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


=== Estrategia 2 - Logistic Regression (MICE) ===
Accuracy: 0.5374149659863946
Macro F1: 0.37659224441833133
[[56 10  0  0  0]
 [14 12  2  3  0]
 [ 7  6  3  5  1]
 [ 4  8  1  7  1]
 [ 1  2  2  1  1]]


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 400 iteration(s) (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=400).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
#Para este caso el mejor es n = 700 pero aun asi no hay una mejoria
rf_2 = RandomForestClassifier(
    n_estimators=700,
    random_state=42
)

rf_2.fit(X2_train, y2_train)
y2_pred_rf = rf_2.predict(X2_test)

print("=== Estrategia 2 - Random Forest (MICE) ===")
print("Accuracy:", accuracy_score(y2_test, y2_pred_rf))
print("Macro F1:", f1_score(y2_test, y2_pred_rf, average="macro"))
print(confusion_matrix(y2_test, y2_pred_rf))


=== Estrategia 2 - Random Forest (MICE) ===
Accuracy: 0.5170068027210885
Macro F1: 0.34856809144421275
[[57  5  2  2  0]
 [15  9  3  4  0]
 [ 5  5  4  7  1]
 [ 2  8  5  5  1]
 [ 1  3  2  0  1]]
